In [ ]:
! pip install -q bertopic datasets gensim

In [ ]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
from bertopic import BERTopic
from umap import UMAP
from gensim.models.poincare import PoincareModel
from nltk.corpus import wordnet as wn
import nltk
import csv
import os
import itertools
from tqdm import tqdm
import random
from gensim.models.poincare import LinkPredictionEvaluation, ReconstructionEvaluation
import pandas as pd
from utils.bench_mark import generate_bench_mark

In [ ]:
OUTPUT_DIR = "outputs"

In [ ]:

dataset = load_dataset("tweet_eval", "offensive")

In [ ]:
# TODO Do a prettier print of the dataset dictionary
dataset

In [ ]:
# do a pretty of sample train dataset
train_ds = dataset["train"]
train_ds[0]

In [ ]:

dataset.set_format(type="pandas")
df = dataset["train"][:]

def label_int2str(row):
    return dataset["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

df["label_name"].value_counts(ascending=True).plot.barh()
plt.xlabel("Number of Samples")
plt.title("Frequency of Classes")
plt.savefig(os.path.join(OUTPUT_DIR, "class_frequency.png"))
plt.show()

In [ ]:

umap_model = UMAP(random_state=42)

topic_model = BERTopic(umap_model=umap_model)

In [ ]:
train_docs = dataset["train"]['text']
train_classes = dataset["train"]['label']
test_docs = dataset["test"]['text']
test_classes = dataset["test"]['label']
valid_docs = dataset["validation"]['text']
valid_classes = dataset["validation"]['label']

In [ ]:
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True, embedding_model="all-MiniLM-L6-v2")
topics, probs = topic_model.fit_transform(train_docs)

In [ ]:
topic_model.get_topic_info()


In [ ]:
topic_model.get_topic(0)

In [ ]:
len(topic_model.get_topics().values())

In [ ]:
embeddings = []
for topic in topic_model.get_topics().values():
    topic_embeddings = []
    for i in range(len(topic) - 1):
        for j in range(i + 1, len(topic)):
            topic_embeddings.append([topic[j][0], topic[i][0]])
    embeddings.append(topic_embeddings)
print(embeddings[0])

In [ ]:

nltk.download('wordnet')

In [ ]:
flat_list = [item for sublist in embeddings for embedding in sublist for item in embedding]

# Get unique words
vocab = list(set(flat_list))
print(f'Voabulary size: {len(vocab)}')
print(f'Sample of vocabulary: {vocab[:5]}')

In [ ]:
relations = []
def get_hypernym_relations(word, relations):
    """Get hypernym relations for a word."""
    for synset in wn.synsets(word):
        for hypernym in synset.hypernyms():
            hyp = [synset.lemmas()[0].name(), hypernym.lemmas()[0].name()]
            if len(hyp) != 0:
              relations.append(hyp)
    return relations

for v in vocab:
  get_hypernym_relations(f'{v}', relations)
print(f'Number of relations: {len(relations)}')

print(relations[:5])

In [ ]:

DATA_DIRECTORY = 'data'
test_size_ratio = 0.2
train_relation = relations[:int(len(relations) * (1 - test_size_ratio))]
test_relation = relations[int(len(relations) * (1 - test_size_ratio)):]
train_relation_file_path = os.path.join(DATA_DIRECTORY, 'train_relation.tsv')
test_relation_file_path = os.path.join(DATA_DIRECTORY, 'test_relation.tsv')
